In [1]:
import os 

os.chdir("../..")

from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq, AutoModelForSequenceClassification
from peft import get_peft_model, LoraConfig, TaskType
import evaluate
import numpy as np
from task1.config import ProjectPaths
import pandas as pd
import torch

paths = ProjectPaths()

# === 3. Set device ===
device = "mps" if torch.backends.mps.is_available() else "cpu"

# === 4. Load and preprocess data ===
def load_datasets(path):
    df = pd.read_csv(path, sep='\t')
    df = df[df['label'].isin(['SUBJ', 'OBJ'])].copy()
    df['label'] = df['label'].map({'OBJ': 0, 'SUBJ': 1})
    df = df[['sentence', 'label']]
    return Dataset.from_pandas(df)

train_dataset = load_datasets(paths.italian_data_dir / "train_it.tsv")
val_dataset   = load_datasets(paths.italian_data_dir / "dev_it.tsv")
test_dataset  = load_datasets(paths.italian_data_dir / "dev_test_it.tsv")
competition_test_dataset = load_datasets(paths.italian_data_dir / "test_it_labeled.tsv")

W0615 11:19:09.821000 8624 Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [2]:
model_name = "Musixmatch/umberto-commoncrawl-cased-v1"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at Musixmatch/umberto-commoncrawl-cased-v1 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
def tokenize_fn(examples):
    return tokenizer(
        examples["sentence"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

train_dataset = train_dataset.map(tokenize_fn, batched=True)
val_dataset = val_dataset.map(tokenize_fn, batched=True)
test_dataset = test_dataset.map(tokenize_fn, batched=True)
competition_test_dataset = competition_test_dataset.map(tokenize_fn, batched=True)

train_dataset = train_dataset.rename_column("label", "labels")
val_dataset = val_dataset.rename_column("label", "labels")
test_dataset = test_dataset.rename_column("label", "labels")
competition_test_dataset = competition_test_dataset.rename_column("label", "labels")

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
competition_test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/1613 [00:00<?, ? examples/s]

Map:   0%|          | 0/667 [00:00<?, ? examples/s]

Map:   0%|          | 0/462 [00:00<?, ? examples/s]

Map:   0%|          | 0/299 [00:00<?, ? examples/s]

In [4]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    task_type=TaskType.SEQ_CLS,
    target_modules=["query", "key", "value", "dense"] 
)

model = get_peft_model(model, lora_config).to(device)

In [5]:
f1 = evaluate.load("f1")
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1.compute(predictions=preds, references=labels, average="macro")["f1"],
        "precision": precision.compute(predictions=preds, references=labels, average="macro")["precision"],
        "recall": recall.compute(predictions=preds, references=labels, average="macro")["recall"],
    }

In [6]:
# === 8. TrainingArguments ===
training_args = TrainingArguments(
    output_dir="./results/italian-lora",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
)

In [7]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

# === 10. Train ===
trainer.train()

trainer.save_model("./results/italian-lora/final_checkpoint")

# === 11. Evaluate on test set ===
trainer.evaluate(eval_dataset=test_dataset)

  0%|          | 0/4040 [00:00<?, ?it/s]

d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 0.6477, 'grad_norm': 1.2946667671203613, 'learning_rate': 4.987623762376238e-05, 'epoch': 0.02}
{'loss': 0.635, 'grad_norm': 2.679760217666626, 'learning_rate': 4.975247524752475e-05, 'epoch': 0.05}
{'loss': 0.5162, 'grad_norm': 2.4853668212890625, 'learning_rate': 4.9628712871287133e-05, 'epoch': 0.07}
{'loss': 0.5719, 'grad_norm': 2.0953242778778076, 'learning_rate': 4.950495049504951e-05, 'epoch': 0.1}
{'loss': 0.547, 'grad_norm': 1.3017030954360962, 'learning_rate': 4.938118811881188e-05, 'epoch': 0.12}
{'loss': 0.5817, 'grad_norm': 2.492706537246704, 'learning_rate': 4.925742574257426e-05, 'epoch': 0.15}
{'loss': 0.6319, 'grad_norm': 1.0773237943649292, 'learning_rate': 4.913366336633663e-05, 'epoch': 0.17}
{'loss': 0.4662, 'grad_norm': 2.4209187030792236, 'learning_rate': 4.9009900990099014e-05, 'epoch': 0.2}
{'loss': 0.6549, 'grad_norm': 1.5622074604034424, 'learning_rate': 4.888613861386139e-05, 'epoch': 0.22}
{'loss': 0.5656, 'grad_norm': 1.5578254461288452, 'learning

  0%|          | 0/167 [00:00<?, ?it/s]

{'eval_loss': 0.47414451837539673, 'eval_accuracy': 0.7571214392803598, 'eval_f1_macro': 0.6790297283809795, 'eval_precision': 0.685347261434218, 'eval_recall': 0.6740977747031016, 'eval_runtime': 32.6237, 'eval_samples_per_second': 20.445, 'eval_steps_per_second': 5.119, 'epoch': 1.0}


...

d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'train_runtime': 5106.0853, 'train_samples_per_second': 3.159, 'train_steps_per_second': 0.791, 'train_loss': 0.3716635705654205, 'epoch': 10.0}


d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


  0%|          | 0/116 [00:00<?, ?it/s]

{'eval_loss': 0.7847893834114075,
 'eval_accuracy': 0.829004329004329,
 'eval_f1_macro': 0.7650792016117094,
 'eval_precision': 0.8044846577498033,
 'eval_recall': 0.7444096182634731,
 'eval_runtime': 23.1998,
 'eval_samples_per_second': 19.914,
 'eval_steps_per_second': 5.0,
 'epoch': 10.0}

In [8]:
trainer.evaluate(eval_dataset=competition_test_dataset)

d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


  0%|          | 0/75 [00:00<?, ?it/s]

{'eval_loss': 1.005687952041626,
 'eval_accuracy': 0.7859531772575251,
 'eval_f1_macro': 0.748792270531401,
 'eval_precision': 0.7837545337545337,
 'eval_recall': 0.7361029984423676,
 'eval_runtime': 15.3739,
 'eval_samples_per_second': 19.449,
 'eval_steps_per_second': 4.878,
 'epoch': 10.0}